In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from keras.optimizers import RMSprop
from keras.models import Sequential, load_model
from keras.layers import Dense, BatchNormalization, Dropout
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


***
# =-=-=-=-=-=-=-=-=-= SETAR DATA E RODOVIA =-=-=-=-=-=-=-=-=-=

In [2]:
train_data = '2020-01-07'
eval_data  = '2020-01-07'
rodovia = 'dutra'

***
* Leituras geradas a partir da base de dados

https://qastack.com.br/stats/11859/what-is-the-difference-between-multiclass-and-multilabel-problem


assistir: https://www.youtube.com/watch?v=d8U7ygZ48Sc&t=24s

In [3]:
df_train = pd.read_csv('../outputs_' + train_data + '/' + rodovia + '/leituras_' + rodovia + '_all_days.csv', delimiter=',')
df_train

,Id,Rodovia,State,KM,7_days_nro_smartphones,7_days_esp_00h_06hs,7_days_esp_06_12hs,7_days_esp_12_18,7_days_esp_18_24hs,7_days_total,...,1_day_esp_12_18,1_day_esp_18_24hs,1_day_total,day_nro_smartphones,esp_00h_06hs,esp_06_12hs,esp_12_18,esp_18_24hs,day_total,Fator_X
0,1,Presidente Dutra,SP,232,1,0,0,0,0,3,...,0,0,42,14,0,0,0,0,42,3.0
1,2,Presidente Dutra,SP,231,6,0,0,0,0,18,...,0,0,12,31,0,0,0,0,93,3.0
2,3,Presidente Dutra,SP,230,2,0,0,0,0,6,...,0,1,1,6,0,0,2,0,2,3.0
3,4,Presidente Dutra,SP,229,96,0,1,1,0,2,...,0,0,300,101,0,0,0,0,303,3.0
4,5,Presidente Dutra,SP,228,5,0,2,1,1,4,...,1,1,6,8,0,1,2,0,3,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
398,399,Presidente Dutra,RJ,168,3,0,0,0,0,9,...,0,1,1,1,0,0,0,1,1,3.0
399,400,Presidente Dutra,RJ,167,8,0,1,1,1,3,...,1,0,1,6,0,1,2,2,5,3.0
400,401,Presidente Dutra,RJ,166,3,0,1,0,0,1,...,1,1,5,1,1,0,2,1,4,3.0
401,402,Presidente Dutra,RJ,165,0,0,0,0,0,0,...,0,0,24,1,0,0,1,0,1,3.0


***
# Definição de Features e Target

* Features

Os dados dos ultimos 7 dias

In [4]:
features = ['7_days_esp_00h_06hs', '7_days_esp_06_12hs', '7_days_esp_12_18', '7_days_esp_18_24hs', \
            '6_days_esp_00h_06hs', '6_days_esp_06_12hs', '6_days_esp_12_18', '6_days_esp_18_24hs', \
            '5_days_esp_00h_06hs', '5_days_esp_06_12hs', '5_days_esp_12_18', '5_days_esp_18_24hs', \
            '4_days_esp_00h_06hs', '4_days_esp_06_12hs', '4_days_esp_12_18', '4_days_esp_18_24hs', \
            '3_days_esp_00h_06hs', '3_days_esp_06_12hs', '3_days_esp_12_18', '3_days_esp_18_24hs', \
            '2_days_esp_00h_06hs', '2_days_esp_06_12hs', '2_days_esp_12_18', '2_days_esp_18_24hs', \
            '1_day_esp_00h_06hs' , '1_day_esp_06_12hs' , '1_day_esp_12_18' , '1_day_esp_18_24hs']

In [5]:
X = df_train[features]
X

,7_days_esp_00h_06hs,7_days_esp_06_12hs,7_days_esp_12_18,7_days_esp_18_24hs,6_days_esp_00h_06hs,6_days_esp_06_12hs,6_days_esp_12_18,6_days_esp_18_24hs,5_days_esp_00h_06hs,5_days_esp_06_12hs,...,3_days_esp_12_18,3_days_esp_18_24hs,2_days_esp_00h_06hs,2_days_esp_06_12hs,2_days_esp_12_18,2_days_esp_18_24hs,1_day_esp_00h_06hs,1_day_esp_06_12hs,1_day_esp_12_18,1_day_esp_18_24hs
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,1,1,0,1,0,0,0,0,0,...,1,0,1,0,0,1,0,0,0,0
4,0,2,1,1,1,0,0,2,0,1,...,2,1,1,0,2,1,2,2,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
398,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
399,0,1,1,1,0,0,0,0,0,1,...,0,1,0,0,2,0,0,0,1,0
400,0,1,0,0,1,0,0,2,0,0,...,0,2,0,0,0,2,1,2,1,1
401,0,0,0,0,0,0,0,1,0,0,...,2,1,0,0,0,0,0,0,0,0


* Target

In [6]:
target = ['esp_00h_06hs', 'esp_06_12hs', 'esp_12_18', 'esp_18_24hs']

In [7]:
y = df_train[target]

In [8]:
scaler = StandardScaler().fit(X)
X = scaler.transform(X)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=0.4,
                                                    random_state=42)

* Model

* best network

In [10]:
model = Sequential()
model.add(Dense(100, activation='relu', input_shape = (X.shape[1],)))
model.add(Dense(100, activation='relu'))
model.add(Dense(len(y.columns)))  # output layer

# model.compile(optimizer='sgd',  loss='categorical_crossentropy', metrics=['accuracy'])
opt = RMSprop(lr=0.0001, decay=1e-6)
model.compile(optimizer='adam',  loss='binary_crossentropy', metrics=['accuracy'])
# model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

* Fit

In [11]:
early_stopping_monitor = EarlyStopping(monitor='val_loss',    # allows you to specify the performance measure to monitor in order to end training
                                       patience=100,          # número de épocas sem melhora após o qual o treinamento será interrompido
                                       mode='auto',           # No modo `min`, o treinamento parará quando a quantidade monitorado parou de diminuir; em `max`modo vai parar quando a quantidade monitorado parou de aumentar
                                       baseline=None,         # O treinamento será interrompido se o modelo não mostrar melhora sobre a linha de base
                                       restore_best_weights=True) # para restaurar pesos de modelo de a época com o melhor valor da quantidade monitorada
model.fit(X_train,
          y_train,
          epochs=1000,
          verbose=0,
          validation_data=(X_test, y_test),
          use_multiprocessing=True,
          callbacks=[early_stopping_monitor]
          )

In [12]:
# model.get_config()

In [13]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 100)               2900      
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 404       
Total params: 13,404
Trainable params: 13,404
Non-trainable params: 0
_________________________________________________________________


* Predict

In [14]:
train_preds = model.predict(X_test)

* Metrics

* Evaluation

In [15]:
scores_test = model.evaluate(X_test, y_test)
print("\n%s: %.2f%%" % (model.metrics_names[0], scores_test[0]*100))
print("\n%s: %.2f%%" % (model.metrics_names[1], scores_test[1]*100))

162/162 [==============================] - 0s 31us/step

loss: -22.70%

accuracy: 93.98%


* r2_score - Best possible score is 1.0

In [16]:
r2_score(y_test, train_preds, multioutput='raw_values')

array([ 0.2439982 , -1.31155021,  0.54830766,  0.31754834])

***

* Save Model

In [17]:
model.save('../model/' + rodovia + '/dutra_model_dl.h5')

***
# Make Predictions

* Load CSV, set Features and Target

In [18]:
df_eval = pd.read_csv('../outputs_' + eval_data + '/' + rodovia + '/leituras_' + rodovia + '_all_days.csv', delimiter=',')
X_eval = df_eval[features]
y_eval = df_eval[target]

In [19]:
scaler_eval = StandardScaler().fit(X_eval)
X_eval = scaler_eval.transform(X_eval)

* Load Model

In [20]:
loaded_model = load_model('../model/' + rodovia + '/dutra_model_dl.h5')

* Predction

In [21]:
eval_preds = loaded_model.predict(X_eval)

* Evaluation

In [22]:
scores_preds = loaded_model.evaluate(X_eval, y_eval)
print("\n%s: %.2f%%" % (loaded_model.metrics_names[0], scores_preds[0]*100))
print("\n%s: %.2f%%" % (loaded_model.metrics_names[1], scores_preds[1]*100))

403/403 [==============================] - 0s 72us/step

loss: 8.24%

accuracy: 94.91%


* r2_score - Best possible score is 1.0

In [23]:
r2 = r2_score(y_eval, eval_preds, multioutput='raw_values')
r2
# r2.mean()

array([-0.17554761, -1.47529486,  0.57655527,  0.28468421])

* Save predicted CSV

In [24]:
predicted_reads = pd.DataFrame({
                                 'Id'           : df_eval['Id'].astype(str),
                                 'state'        : df_eval['State'],
                                 'km'           : df_eval['KM'].astype(str),
                                 'esp_00h_06hs' : abs(eval_preds[:, 0].astype(int)),
                                 'esp_06_12hs'  : abs(eval_preds[:, 1].astype(int)),
                                 'esp_12_18'    : abs(eval_preds[:, 2].astype(int)),
                                 'esp_18_24hs'  : abs(eval_preds[:, 3].astype(int))
                               })
predicted_reads.to_csv('../outputs_' + eval_data + '/' + rodovia + '/predicted_' + rodovia + '_ml.csv', index=False)
print(predicted_reads.to_string())
# predicted_reads

      Id state   km  esp_00h_06hs  esp_06_12hs  esp_12_18  esp_18_24hs
0      1    SP  232             0            0          0            0
1      2    SP  231             0            0          0            0
2      3    SP  230             0            0          0            0
3      4    SP  229             0            0          0            0
4      5    SP  228             0            2          1            1
5      6    SP  227             0            0          0            0
6      7    SP  226             0            0          0            0
7      8    SP  225             0            0          0            0
8      9    SP  224             0            0          0            0
9     10    SP  223             5            8          4            3
10    11    SP  222             0            0          0            0
11    12    SP  221             0            0          0            0
12    13    SP  220             0            0          0            0
13    

In [25]:
predicted_reads.describe()

,esp_00h_06hs,esp_06_12hs,esp_12_18,esp_18_24hs
count,403.000000,403.000000,403.000000,403.000000
mean,0.066998,0.099256,0.084367,0.081886
std,0.486615,0.634869,0.562359,0.509360
min,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000
max,6.000000,8.000000,6.000000,7.000000


In [26]:
predicted_reads.iloc[9]

Id               10
state            SP
km              223
esp_00h_06hs      5
esp_06_12hs       8
esp_12_18         4
esp_18_24hs       3
Name: 9, dtype: object

In [27]:
df_train.iloc[9][-4:]

esp_12_18      15
esp_18_24hs    23
day_total      66
Fator_X         3
Name: 9, dtype: object

In [28]:
df_eval.iloc[9][-4:]

esp_12_18      15
esp_18_24hs    23
day_total      66
Fator_X         3
Name: 9, dtype: object

***

In [29]:
predicted_reads.iloc[392]

Id              393
state            RJ
km              174
esp_00h_06hs      6
esp_06_12hs       6
esp_12_18         6
esp_18_24hs       7
Name: 392, dtype: object

In [30]:
df_train.iloc[392][-4:]

esp_12_18       6
esp_18_24hs     7
day_total      21
Fator_X         3
Name: 392, dtype: object

In [31]:
df_eval.iloc[392][-4:]

esp_12_18       6
esp_18_24hs     7
day_total      21
Fator_X         3
Name: 392, dtype: object

***

In [32]:
predicted_reads.iloc[358]

Id              359
state            RJ
km              208
esp_00h_06hs      0
esp_06_12hs       0
esp_12_18         0
esp_18_24hs       0
Name: 358, dtype: object

In [33]:
df_train.iloc[358][-4:]

esp_12_18       0
esp_18_24hs     0
day_total      12
Fator_X         3
Name: 358, dtype: object

In [34]:
df_eval.iloc[358][-4:]

esp_12_18       0
esp_18_24hs     0
day_total      12
Fator_X         3
Name: 358, dtype: object